### 使用 gpxpy 套件解析 gpx

GPX
 └─ trk (Track，軌跡)
      └─ trkseg (Track Segment，路段)
           └─ trkpt (Track Point，座標點)
                ├─ lat        (緯度，必填屬性)
                ├─ lon        (經度，必填屬性)
                ├─ ele       （高度／海拔，選填屬性）
                ├─ time      （時間戳，ISO 8601，選填屬性）
                ├─ speed     （速度，選填屬性，單位多為 m/s）
                ├─ hr        （心率，選填屬性，單位 bpm）
                ├─ course    （方向角，選填屬性，單位度）
                └─ extensions（廠商自定義欄位，如溫度、踏頻、壓力等）

In [ ]:
# 大概觀察一下 gpx 的資訊

import gpxpy


def print_gpx_tree(gpx_path):
    with open(gpx_path, "r", encoding="utf-8") as f:
        gpx = gpxpy.parse(f)

    print(gpx_path)
    for i, track in enumerate(gpx.tracks):
        print(f"  ├─ Track {i+1}: {track.name or '(no name)'}")
        for j, segment in enumerate(track.segments):
            print(f"  │   ├─ Segment {j+1} (points: {len(segment.points)})")
            for k, point in enumerate(segment.points[:]):
                print(
                    f"  │   │   ├─ Point {k+1}: ({point.latitude:.5f}, {point.longitude:.5f}), time: {point.time}"
                )
            if len(segment.points) > 5:
                print(f"  │   │   └─ ... ({len(segment.points)} points total)")
    for i, route in enumerate(gpx.routes):
        print(
            f"  ├─ Route {i+1}: {route.name or '(no name)'} (points: {len(route.points)})"
        )
    for i, waypoint in enumerate(gpx.waypoints):
        print(
            f"  ├─ Waypoint {i+1}: {waypoint.name or '(no name)'} ({waypoint.latitude:.5f}, {waypoint.longitude:.5f})"
        )


# 用法：gpx_path 改為要解析的 gpx 路經
gpx_path = r"123.gpx"
print_gpx_tree(gpx_path)

### 轉檔 GPX →  GeoPandas GeoDataFrame
GeoPandas = pandas DataFrame + geometry 欄位（存 shapely 物件） + 空間索引功能

方便後續地理查詢、繪圖、空間運算...等 GIS 分析

In [ ]:
# 將 gpx 轉檔為 geodataframe 格式，並加入 geometry 欄位，方便讓 shapely 套件進行空間分析
import gpxpy
import geopandas
from shapely.geometry import Point


def gpx_to_geodataframe(gpx_path):
    with open(gpx_path, "r", encoding="utf-8") as f:
        gpx = gpxpy.parse(f)

    rows = []
    for track in gpx.tracks:
        for seg_idx, segment in enumerate(track.segments):
            for pt_idx, p in enumerate(segment.points):
                rows.append(
                    {
                        "track_name": track.name,
                        "segment_index": seg_idx,
                        "point_index": pt_idx,
                        "lon": p.longitude,  # 經度，單位：°
                        "lat": p.latitude,  # 緯度，單位：°
                        "ele": p.elevation,  # 高程，單位：m
                        "time": p.time,  # Timestamp 時間戳記且可能帶有時區
                        "geometry": Point(
                            p.longitude, p.latitude
                        ),  # 注意！ shapely 預設 (x, y) = (lon, lat)，此時單位：°
                    }
                )
    # 建立時指定座標系統是 WGS84 (單位：°)(EPSG 編號 4326)
    gdf = geopandas.GeoDataFrame(rows, geometry="geometry", crs="EPSG:4326")

    # 投影成 TWD97 TM2 (單位：m)，再回傳(EPSG 編號 3826),不然無法計算平面距離(誤差會很大)
    return gdf.to_crs(epsg=3826)


# 用法：gpx_path 改為要解析的 gpx 路經
gpx_path = r"123.gpx"
gdf = gpx_to_geodataframe(gpx_path)
print(gdf.head())
print(gdf.crs)

### 轉檔 GeoDataFrame → CSV

In [ ]:
# 儲存為 CSV

from pathlib import Path

csv_path = str(Path(gpx_path).with_suffix(".csv"))
csv = gdf.to_csv(csv_path, index=False, encoding="utf-8-sig")
print("✔️ 儲存為 CSV")

### CSV → Folium 地圖顯示

In [ ]:
import folium
import pandas

# 用法：自己改路徑名稱
csv_path = "123.csv"
df = pandas.read_csv(csv_path, encoding="utf-8")

lat_col = next(col for col in df.columns if col.startswith("lat"))
lon_col = next(col for col in df.columns if col.startswith("lon"))
coords = list(zip(df[lat_col], df[lon_col]))

m = folium.Map(location=coords[0], zoom_start=14)
folium.PolyLine(coords, color="blue", weight=3, tooltip="Track").add_to(m)
folium.Marker(coords[0], icon=folium.Icon(color="green"), tooltip="Start").add_to(m)
folium.Marker(coords[-1], icon=folium.Icon(color="red"), tooltip="End").add_to(m)


# 變數名稱 -> 直接顯示在 Jupyter Notebook
m